# <center> Régler les hyperparamètres avec Keras Tuner </center>

**Source :** https://www.tensorflow.org/tutorials/keras/keras_tuner?hl=fr

Le Keras Tuner est une bibliothèque qui vous aide à choisir l'ensemble optimal d'hyperparamètres pour votre programme TensorFlow. Le processus de sélection du bon ensemble d'hyperparamètres pour votre application d'apprentissage automatique (ML) est appelé *réglage d'hyperparamètres* ou *hyperréglage*.

Les hyperparamètres sont les variables qui régissent le processus de formation et la topologie d'un modèle ML. Ces variables restent constantes tout au long du processus de formation et ont un impact direct sur les performances de votre programme ML. Les hyperparamètres sont de deux types :
- **Hyperparamètres de modèle** qui influencent la sélection du modèle, tels que le nombre et la largeur des couches masquées

- **Hyperparamètres d'algorithme** qui influencent la vitesse et la qualité de l'algorithme d'apprentissage, tels que le taux d'apprentissage pour Stochastic Gradient Descent (SGD) et le nombre de voisins les plus proches pour un classificateur ak Nearest Neighbors (KNN)



## Importations

In [2]:
import tensorflow as tf
from tensorflow import keras
# pip install -q -U keras-tuner
import keras_tuner as kt

## Jeu de données

In [3]:
(img_train, label_train), (img_test, label_test) = keras.datasets.fashion_mnist.load_data()

4422102/4422102 [==============================] - 0s 0us/step


In [4]:
# Normalize pixel values between 0 and 1
img_train = img_train.astype('float32') / 255.0
img_test = img_test.astype('float32') / 255.0

## Construction du modèle

Lorsque vous créez un modèle pour l'hyperréglage, vous définissez également l'espace de recherche d'hyperparamètres en plus de l'architecture du modèle. Le modèle que vous configurez pour l'hyperréglage s'appelle un *hypermodèle* .

Vous pouvez définir un hypermodèle selon deux approches :

En utilisant une fonction de création de modèle
En sous-classant la classe `HyperModel` de l'API Keras Tuner.

Dans ce didacticiel, vous utilisez une fonction de générateur de modèles pour définir le modèle de classification d'images. La fonction de générateur de modèles renvoie un modèle compilé et utilise des hyperparamètres que vous définissez en ligne pour ajuster le modèle.

In [5]:
def model_builder(hp):
  model = keras.Sequential()
  model.add(keras.layers.Flatten(input_shape=(28, 28)))

  # Tune the number of units in the first Dense layer
  # Choose an optimal value between 32-512
  hp_units = hp.Int('units', min_value=32, max_value=512, step=32)
  model.add(keras.layers.Dense(units=hp_units, activation='relu'))
  model.add(keras.layers.Dense(10))

  # Tune the learning rate for the optimizer
  # Choose an optimal value from 0.01, 0.001, or 0.0001
  hp_learning_rate = hp.Choice('learning_rate', values=[1e-2, 1e-3, 1e-4])

  model.compile(optimizer=keras.optimizers.Adam(learning_rate=hp_learning_rate),
                loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                metrics=['accuracy'])

  return model

## Instanciez le tuner et effectuez un hypertuning

In [6]:
tuner = kt.Hyperband(model_builder,
                     objective='val_accuracy',
                     max_epochs=10,
                     factor=3,
                     directory='my_dir',
                     project_name='intro_to_kt')

In [7]:
# création d'un rappel
stop_early = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)

In [8]:
# rechercher les hyperparamètres
tuner.search(img_train, label_train, epochs=50, validation_split=0.2, callbacks=[stop_early])

# Get the optimal hyperparameters
best_hps=tuner.get_best_hyperparameters(num_trials=1)[0]

print(f"""
The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is {best_hps.get('units')} and the optimal learning rate for the optimizer
is {best_hps.get('learning_rate')}.
""")

Trial 30 Complete [00h 02m 23s]
val_accuracy: 0.8847500085830688

Best val_accuracy So Far: 0.8924999833106995
Total elapsed time: 00h 24m 01s

The hyperparameter search is complete. The optimal number of units in the first densely-connected
layer is 416 and the optimal learning rate for the optimizer
is 0.001.



## Entraînement du modèle

In [9]:
# Build the model with the optimal hyperparameters and train it on the data for 50 epochs
model = tuner.hypermodel.build(best_hps)
history = model.fit(img_train, label_train, epochs=50, validation_split=0.2)

val_acc_per_epoch = history.history['val_accuracy']
best_epoch = val_acc_per_epoch.index(max(val_acc_per_epoch)) + 1
print('Best epoch: %d' % (best_epoch,))

Epoch 1/50
1500/1500 [==============================] - 14s 9ms/step - loss: 0.4931 - accuracy: 0.8252 - val_loss: 0.4065 - val_accuracy: 0.8561
Epoch 2/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3699 - accuracy: 0.8657 - val_loss: 0.3658 - val_accuracy: 0.8652
Epoch 3/50
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3326 - accuracy: 0.8779 - val_loss: 0.3330 - val_accuracy: 0.8764
Epoch 4/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.3087 - accuracy: 0.8860 - val_loss: 0.3292 - val_accuracy: 0.8804
Epoch 5/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.2879 - accuracy: 0.8922 - val_loss: 0.3487 - val_accuracy: 0.8750
Epoch 6/50
1500/1500 [==============================] - 13s 9ms/step - loss: 0.2734 - accuracy: 0.8988 - val_loss: 0.3321 - val_accuracy: 0.8779
Epoch 7/50
1500/1500 [==============================] - 13s 8ms/step - loss: 0.2564 - accuracy: 0.9031 - val_loss: 0.3038 - val_ac

## Réentraînement avec le nombre optimale d'epochs

In [10]:
hypermodel = tuner.hypermodel.build(best_hps)

# Retrain the model
hypermodel.fit(img_train, label_train, epochs=best_epoch, validation_split=0.2)

Epoch 1/18
1500/1500 [==============================] - 13s 8ms/step - loss: 0.4941 - accuracy: 0.8229 - val_loss: 0.3937 - val_accuracy: 0.8563
Epoch 2/18
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3685 - accuracy: 0.8665 - val_loss: 0.4190 - val_accuracy: 0.8432
Epoch 3/18
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3299 - accuracy: 0.8790 - val_loss: 0.3552 - val_accuracy: 0.8703
Epoch 4/18
1500/1500 [==============================] - 12s 8ms/step - loss: 0.3047 - accuracy: 0.8879 - val_loss: 0.3322 - val_accuracy: 0.8795
Epoch 5/18
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2832 - accuracy: 0.8941 - val_loss: 0.3582 - val_accuracy: 0.8788
Epoch 6/18
1500/1500 [==============================] - 12s 8ms/step - loss: 0.2705 - accuracy: 0.8999 - val_loss: 0.3210 - val_accuracy: 0.8870
Epoch 7/18
1500/1500 [==============================] - 13s 9ms/step - loss: 0.2577 - accuracy: 0.9038 - val_loss: 0.3246 - val_ac

## Evaluation : 

In [11]:
eval_result = hypermodel.evaluate(img_test, label_test)
print("[test loss, test accuracy]:", eval_result)

313/313 [==============================] - 2s 4ms/step - loss: 0.3866 - accuracy: 0.8831
[test loss, test accuracy]: [0.3865816593170166, 0.8830999732017517]
